# Project Group - 25

Members: Yun-An LIN (5841682)</br>
Rohan Menezes (5850908)</br>
John Kuttikat (5765382)</br>
Muhammad Rizki Ziarieputra (5848113)</br>
Ian Trout (5851483)</br>

# Research Objective

*Did COVID impact vessel operations? </br> An analysis of several vessel/port factors, including geospatial representation.*

# Contribution Statement

**Yun-An LIN**: Data for CPPI (both data frames and visualization)

**Rohan Menezes**: Streamlit Data Visualisation, Data filtering

**John Kuttikat**: Data manipulation and filtering, compilation of code, covid data
    
**Muhammad Rizki Ziarieputra**: Data gathering and analyzing port & covid data.

**Ian Trout**: Data research, Peak/Valley calculations, world data visualisation, narrative

# Data Used

1. Covid data (https://data.humdata.org/dataset/coronavirus-covid-19-cases-and-deaths) 

2. Port data (https://unctadstat.unctad.org/wds/TableViewer/tableView.aspx?ReportId=170027)

3. Port Calls data (https://unctadstat.unctad.org/wds/TableViewer/tableView.aspx?ReportId=194890)

4. The Container Port PERFORMANCE INDEX (CPPI) from World Bank Group (https://thedocs.worldbank.org/en/doc/66e3aa5c3be4647addd01845ce353992-0190062022/original/Container-Port-Performance-Index-2021.pdf)

# Section 1: Importing Libraries

Here, we have added the necessary libraries required for the proper functioning of the program.

In [1]:
import pandas as pd
import chardet
from plotly.offline import init_notebook_mode
import numpy as np
from numpy import mean
from numpy import std
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import itertools
from datetime import datetime, timedelta, date #CHANGED THIS due to attribute error down in the peaks_valleys work
from dateutil.relativedelta import relativedelta #added this one to get look at peaks/valleys that are next to each other
import math
import scipy
from scipy.signal import find_peaks
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
from scipy.stats import spearmanr
import streamlit as st


init_notebook_mode(connected=True)
pio.renderers.default = "plotly_mimetype+notebook"

# Section 2: Importing Data

Firstly, all the data are organized here for processing.</br></br>
Datas used:
1. Port Data - This data set includes all the data regarding the  Median time in port, etc for various vessel types around the world. Port data from UNICSTAT on a country level, giving tonnage, median time in port, and other information (from 2018 to 2022)
2. port calls Data - This data set includes all the data regarding the number of port call for each vessel type around the world.
2. Cocid Data - This data set provide information on the number of covid cases around the world. Covid data from the WHO on country level, giving cases, hospitalizations and casualties per day (absolute and cumulative)
3. Geo Data  - GeoJSON file of all the countries in the world
4. Port performance data - Port peformance index data for several ports within a country (data ranging from 2020 to 2021)


In [2]:
# port data
portcalls_file_path = "Data/Maritime data/US_PortCalls_S_ST202209220924_v1.csv"
with open(portcalls_file_path, 'rb') as rawdata:    
    result = chardet.detect(rawdata.read(100000))
result

df_port = pd.read_csv(portcalls_file_path,encoding='utf-8') # Reading the port data into 'df_ports'

# port calls data
portcalls_file_path_new = "Data/Maritime data/US_PortCallsArrivals_S_ST202209220927_v1.csv"
df_port_calls = pd.read_csv(portcalls_file_path_new ,encoding='utf-8')

# covid Data
covid_file_path = "Data\COVID data\WHO-COVID-19-global-data.csv"
df_covid = pd.read_csv(covid_file_path) # Reading the covid data into 'df_covid'

df_geo = gpd.read_file("Data/countries.geojson") # geojson file
df_geo.rename(columns = {'ADMIN': 'Location', }, inplace=True)

# Port Performance Data
performance_path = "Data/The productivity of the ports/Container-Port-Performance-Index-2021 copy.csv"
df_performance = pd.read_csv(performance_path)





## Viewing the initial database structures before any processing or manipulation.</br>
### This gives us an insight of the database regarding the data present so that we can alter the data set to our requirement. 

#### 1. <u>Port Data

In [3]:
df_port.head()

,Period,Period Label,Year,Frequency,Frequency Label,Economy,Economy Label,CommercialMarket,CommercialMarket Label,Median time in port (days),...,Maximum size (GT) of vessels,Maximum size (GT) of vessels Footnote,Average cargo carrying capacity (dwt) per vessel,Average cargo carrying capacity (dwt) per vessel Footnote,Maximum cargo carrying capacity (dwt) of vessels,Maximum cargo carrying capacity (dwt) of vessels Footnote,Average container carrying capacity (TEU) per container ship,Average container carrying capacity (TEU) per container ship Footnote,Maximum container carrying capacity (TEU) of container ships,Maximum container carrying capacity (TEU) of container ships Footnote
0,2018S01,S1 2018,2018,S,Semi-annual,0,World,0,All ships,0.97,...,234006,NaN,24074.0,NaN,441561.0,NaN,3526.0,NaN,21413.0,NaN
1,2018S01,S1 2018,2018,S,Semi-annual,0,World,1,Passenger ships,NaN,...,228081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018S01,S1 2018,2018,S,Semi-annual,0,World,2,Liquid bulk carriers,0.94,...,234006,NaN,26871.0,NaN,441561.0,NaN,NaN,NaN,NaN,NaN
3,2018S01,S1 2018,2018,S,Semi-annual,0,World,3,Container ships,0.69,...,217673,NaN,NaN,NaN,NaN,NaN,3526.0,NaN,21413.0,NaN
4,2018S01,S1 2018,2018,S,Semi-annual,0,World,4,Dry breakbulk carriers,1.12,...,91784,NaN,7413.0,NaN,138743.0,NaN,NaN,NaN,NaN,NaN


#### 2. <u>Port Calls Data

In [4]:
df_port_calls.head()

,Period,Period Label,Year,Frequency,Frequency Label,Economy,Economy Label,CommercialMarket,CommercialMarket Label,Number of port calls,Number of port calls Footnote
0,2018S01,S1 2018,2018,S,Semi-annual,0,World,0,All ships,1984908,NaN
1,2018S01,S1 2018,2018,S,Semi-annual,0,World,1,Passenger ships,1053697,NaN
2,2018S01,S1 2018,2018,S,Semi-annual,0,World,2,Liquid bulk carriers,245147,NaN
3,2018S01,S1 2018,2018,S,Semi-annual,0,World,3,Container ships,226063,NaN
4,2018S01,S1 2018,2018,S,Semi-annual,0,World,4,Dry breakbulk carriers,211031,NaN


#### 3. <u>Covid Data

In [5]:
df_covid.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020/1/3,AF,Afghanistan,EMRO,0,0,0,0
1,2020/1/4,AF,Afghanistan,EMRO,0,0,0,0
2,2020/1/5,AF,Afghanistan,EMRO,0,0,0,0
3,2020/1/6,AF,Afghanistan,EMRO,0,0,0,0
4,2020/1/7,AF,Afghanistan,EMRO,0,0,0,0


#### 4. <u>Geo Data

In [6]:
df_geo.head()

,country,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


#### 5. <u>Performance Index of Ports

In [7]:
df_performance.head()

,Port Name,Administrative Approach Rank,Total\r\nPoints,Statistic Approach Rank,IndeX Value,Economy Label,Year
0,AARHUS,44,-2.036,43,43.000,Denmark,2020
1,ABIDJAN,360,-216.138,359,-93.807,El Salvador's,2020
2,ABIDJAN,228,0.439,217,-5.000,Ivory Coast,2020
3,ACAJUTLA,249,0.640,249,-11.000,Australia,2020
4,ADELAIDE,339,4.546,333,-78.000,Yemen,2020


# Section 3: Data processing
Here, the data sets are trimmed and renamed (for standardization) so that the data can be easily referenced and used later in the program. This makes the data set more readable and easier to work with. In this section, we alter a few columns to help us combine and compare data with other data sets.

## Section 3.1: Data Filtering
In this section all data sets are filtered ,standarized and made available for use in further sections of the program.

### Port Data

Issues with the 'date' column of port data:
1. The data-set only provided data for every 6 months from 2018 to 2022.
2. No 'date' column were provided.
3. Column 'Period' had to be modified so that we can merge port data with the covid data.<br/>
3.1 This was done by running the data through a loop and changing the date format<br/>
3.2 The format was chosen to match with that of the covid data 
4. Various columns had to be dropped to make the database more readable.
5. Columns had to be modified to standarize the columns with the covid data for merging purposes.
6. The 'Period' contains the timeline in the format '2018S01', which refers to the year 2018 and semester 1. This includes the cumulative data from 01.01.2018 to 31.07.2018. This data was converted to the DateTime format, based on the condition above, to match with the other sets of data frame used in this analysis. 

In [8]:
# Firstly, we remove all the unnecessary fields from the data set.
df_port = df_port.drop(columns=[
                            'Period Label','Frequency', 
                            'Frequency Label', 'Economy', 
                            'CommercialMarket', 
                            'Median time in port (days) Footnote',
                            'Average age of vessels Footnote', 
                            'Average size (GT) of vessels Footnote',
                            'Maximum size (GT) of vessels Footnote', 
                            'Average cargo carrying capacity (dwt) per vessel Footnote',
                            'Maximum cargo carrying capacity (dwt) of vessels Footnote',
                            'Average container carrying capacity (TEU) per container ship Footnote',
                            'Maximum container carrying capacity (TEU) of container ships Footnote',
                            'Maximum size (GT) of vessels',
                            'Maximum cargo carrying capacity (dwt) of vessels',
                            'Average container carrying capacity (TEU) per container ship',
                            'Maximum container carrying capacity (TEU) of container ships'
                            ])

# we rename the column name to be easily recognizable.
df_port.rename(columns = {
                    'Economy Label': 'country', 
                    'CommercialMarket Label': 'vessel_type', 
                    'Year': 'year', 
                    'Period':'period', 
                    'Median time in port (days)':'median_time_in_port',
                    'Average age of vessels':'avg_age_of_vessel',
                    'Average size (GT) of vessels':'avg_size_of_vessel',
                    'Average cargo carrying capacity (dwt) per vessel':'avg_dwt'
                    }, inplace=True)

# the time frame in the Period column is modified to match with the date column of the covid data.
date_change=[]
for row in df_port['period']:
    if row == '2018S01' :   
        date_change.append(datetime(2018,7,31))
    elif row == '2018S02':  
         date_change.append(datetime(2019,1,31))
    elif row == '2019S01':  
        date_change.append(datetime(2019,7,31))
    elif row == '2019S02':  
        date_change.append(datetime(2020,1,31))
    elif row == '2020S01':  
        date_change.append(datetime(2020,7,31))
    elif row == '2020S02':  
        date_change.append(datetime(2021,1,31))
    elif row == '2021S01':  
        date_change.append(datetime(2021,7,31))
    elif row == '2021S02':  
        date_change.append(datetime(2022,1,31))
    elif row == '2022S01':  
        date_change.append(datetime(2022,7,31))
    
    else:           
        date_change.append('Not_Rated')

# Adding the new column with the standarized date values.
df_port['date'] = date_change



In [9]:
df_port.head()

,period,year,country,vessel_type,median_time_in_port,avg_age_of_vessel,avg_size_of_vessel,avg_dwt,date
0,2018S01,2018,World,All ships,0.97,18,15222,24074.0,2018-07-31
1,2018S01,2018,World,Passenger ships,NaN,21,8978,NaN,2018-07-31
2,2018S01,2018,World,Liquid bulk carriers,0.94,13,15470,26871.0,2018-07-31
3,2018S01,2018,World,Container ships,0.69,13,38405,NaN,2018-07-31
4,2018S01,2018,World,Dry breakbulk carriers,1.12,19,5455,7413.0,2018-07-31


### Port calls Data


The port calls data is similar to the port data which contains the data of the port calls. The 'Period' had the same problem with 
1.	The port calls data is already on a format of each semester
2.	For analyzing the data, we do some adjustment to the data especially to the column name in order to have a uniformity among the data frames
3.	We also drop the unnecessary data from the data frame to keep the data lean
4.	We finally adjust the date format still in order to have a uniformity in the data frames


In [10]:
# Firstly, we remove all the unnecessary fields from the data set.
df_port_calls = df_port_calls.drop(columns=[
                                        'Period Label', 'Frequency', 
                                        'Frequency Label', 'Economy', 
                                        'CommercialMarket', 'Number of port calls Footnote'
                                        ])

# we rename the column name to be easily recognizable.
df_port_calls.rename(columns = {
                            'Economy Label': 'country', 
                            'CommercialMarket Label': 'vessel_type', 
                            'Year':'year', 
                            'Number of port calls':'num_port_calls', 
                            'Period':'period'
                            }, inplace=True)

# the time frame in the Period column is modified to match with the date column of the covid data.
date_change=[]
for row in df_port_calls['period']:
    if row == '2018S01' :   
        date_change.append(datetime(2018,7,31))
    elif row == '2018S02':  
         date_change.append(datetime(2019,1,31))
    elif row == '2019S01':  
        date_change.append(datetime(2019,7,31))
    elif row == '2019S02':  
        date_change.append(datetime(2020,1,31))
    elif row == '2020S01':  
        date_change.append(datetime(2020,7,31))
    elif row == '2020S02':  
        date_change.append(datetime(2021,1,31))
    elif row == '2021S01':  
        date_change.append(datetime(2021,7,31))
    elif row == '2021S02':  
        date_change.append(datetime(2022,1,31))
    elif row == '2022S01':  
        date_change.append(datetime(2022,7,31))
    
    else:           
        date_change.append('Not_Rated')

# Adding the new column with the standarized date values.
df_port_calls['date'] = date_change

In [11]:
df_port_calls


,period,year,country,vessel_type,num_port_calls,date
0,2018S01,2018,World,All ships,1984908,2018-07-31
1,2018S01,2018,World,Passenger ships,1053697,2018-07-31
2,2018S01,2018,World,Liquid bulk carriers,245147,2018-07-31
3,2018S01,2018,World,Container ships,226063,2018-07-31
4,2018S01,2018,World,Dry breakbulk carriers,211031,2018-07-31
...,...,...,...,...,...,...
15697,2022S01,2022,United Kingdom,Dry breakbulk carriers,7967,2022-07-31
15698,2022S01,2022,United Kingdom,Dry bulk carriers,956,2022-07-31
15699,2022S01,2022,United Kingdom,Roll-on/ roll-off ships,7983,2022-07-31
15700,2022S01,2022,United Kingdom,Liquefied petroleum gas carriers,632,2022-07-31


### Covid Data

1. The columns of the covid data was fisrt renamed for making it easily readable. then, unwanted columns were removed as we had no use of it in our analysis.
2. The 'date' format had to be corrected to match with other data sets so that it can be merged easily.
3. A new data is created called 'df_covid_new' that has the whole data grouped into 6 months. This is because our 'Port Data' had data for every 6 months
4. 'The United Kingdom' had to be standardized to 'United Kingdom' so that it can be compared with other data sets.

In [12]:
df_covid = df_covid.rename({
                        'Date_reported': 'date',
                        'Country': 'country',
                        'New_cases': 'new_cases',
                        'Cumulative_cases': 'cumulative_cases'
                        }, axis=1) 
                        
df_covid = df_covid.drop(labels=[
                            'New_deaths', 
                            'Cumulative_deaths', 
                            'Country_code', 
                            'WHO_region'
                            ], axis=1)

for i in range(len(df_covid)):
    k=df_covid.iloc[i,0].split('/')
    df_covid.iloc[i,0]=datetime(int(k[0]),int(k[1]),int(k[2]))

df_covid_new = (df_covid.groupby(['country', pd.Grouper(key='date', freq='6M')])
        .sum() # gives the max value of the cumulative cases. IMP: To find 'new cases trend', use 'sum()'
        .reset_index())
df_covid_new['country'][df_covid_new['country'] == 'The United Kingdom']='United Kingdom' # standarizes the country name




C:\Users\john_\AppData\Local\Temp\ipykernel_40356\3741742687.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



when grouping is done for the covid data for 6 months, one of the two operations can be done on the data in the  columns 'new_cases' and 'cumulative_cases'. It can be either summed up or the maximum value of the column can be returned. This decision is taken based on the type of comparison that we are looking for. The column 'new_cases' has to be summed up for the period of 6 months. In which case, column 'cumulative_cases' can be neglected as it would have no meaning. Similarly, if we want to compare the cumulative cases for the period of 6 months, the max of column 'cumulative_cases' needs to be analyzed. where, the column 'new_cases' can be neglected as it would have no meaning.

In [13]:
df_covid_new.head()

,country,date,new_cases,cumulative_cases
0,Afghanistan,2020-01-31,0,0
1,Afghanistan,2020-07-31,36628,2080134
2,Afghanistan,2021-01-31,18395,8092870
3,Afghanistan,2021-07-31,92131,14105445
4,Afghanistan,2022-01-31,14986,28667488


### Geo Data

Renaming the Geo data set column to the standard format for merging. This step helps in comparing the geo data with other data sets further in the program

In [14]:
df_geo.rename(columns = {'Location': 'country', }, inplace=True)
df_geo.head()

,country,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


### Performance data

Unnecessary columns are removed for easy interpretation of the data set. Some columns are renamed for stadardization so that they may be easily referenced later in the program.

In [15]:
df_performance=df_performance.drop(labels=[
                            'Administrative Approach Rank',
                            'Total\r\nPoints'
                            ], axis=1)
df_performance.rename(columns = {
                            'Economy Label': 'country', 
                            'Port Name':'port_name', 
                            'Year':'year', 
                            'Statistic Approach Rank':'statistic_approach_rank',
                            'IndeX Value':'index_value' 
                            }, inplace=True)
df_performance.head()

,port_name,statistic_approach_rank,index_value,country,year
0,AARHUS,43,43.000,Denmark,2020
1,ABIDJAN,359,-93.807,El Salvador's,2020
2,ABIDJAN,217,-5.000,Ivory Coast,2020
3,ACAJUTLA,249,-11.000,Australia,2020
4,ADELAIDE,333,-78.000,Yemen,2020


## Section 3.2: Merging of different data sets
In this section, the stadarized data sets from the previous section are combined so that the necessary data are available for data manipulation and data visualization

### Merging Geo data and Port data

In [16]:
geo_port = pd.merge(df_geo, df_port, on = 'country')
geo_port.head()


,country,ISO_A3,geometry,period,year,vessel_type,median_time_in_port,avg_age_of_vessel,avg_size_of_vessel,avg_dwt,date
0,Australia,AUS,"MULTIPOLYGON (((158.86573 -54.74993, 158.83823...",2018S01,2018,All ships,1.49,19,25686,78572.0,2018-07-31
1,Australia,AUS,"MULTIPOLYGON (((158.86573 -54.74993, 158.83823...",2018S01,2018,Passenger ships,NaN,27,5105,NaN,2018-07-31
2,Australia,AUS,"MULTIPOLYGON (((158.86573 -54.74993, 158.83823...",2018S01,2018,Liquid bulk carriers,1.34,7,23585,40187.0,2018-07-31
3,Australia,AUS,"MULTIPOLYGON (((158.86573 -54.74993, 158.83823...",2018S01,2018,Container ships,1.19,12,46778,NaN,2018-07-31
4,Australia,AUS,"MULTIPOLYGON (((158.86573 -54.74993, 158.83823...",2018S01,2018,Dry breakbulk carriers,1.69,12,15417,21345.0,2018-07-31


### Merging of port data and the covid data

In [17]:
port_covid = pd.merge(df_port, df_covid_new, on=['country','date'], how='outer')
port_covid.head()

,period,year,country,vessel_type,median_time_in_port,avg_age_of_vessel,avg_size_of_vessel,avg_dwt,date,new_cases,cumulative_cases
0,2018S01,2018.0,World,All ships,0.97,18.0,15222.0,24074.0,2018-07-31,NaN,NaN
1,2018S01,2018.0,World,Passenger ships,NaN,21.0,8978.0,NaN,2018-07-31,NaN,NaN
2,2018S01,2018.0,World,Liquid bulk carriers,0.94,13.0,15470.0,26871.0,2018-07-31,NaN,NaN
3,2018S01,2018.0,World,Container ships,0.69,13.0,38405.0,NaN,2018-07-31,NaN,NaN
4,2018S01,2018.0,World,Dry breakbulk carriers,1.12,19.0,5455.0,7413.0,2018-07-31,NaN,NaN


### Merging of port calls data and the covid data

In [18]:
portcalls_covid = pd.merge(df_port_calls, df_covid_new, on=['country','date'], how='outer')
portcalls_covid.head()

,period,year,country,vessel_type,num_port_calls,date,new_cases,cumulative_cases
0,2018S01,2018.0,World,All ships,1984908.0,2018-07-31,NaN,NaN
1,2018S01,2018.0,World,Passenger ships,1053697.0,2018-07-31,NaN,NaN
2,2018S01,2018.0,World,Liquid bulk carriers,245147.0,2018-07-31,NaN,NaN
3,2018S01,2018.0,World,Container ships,226063.0,2018-07-31,NaN,NaN
4,2018S01,2018.0,World,Dry breakbulk carriers,211031.0,2018-07-31,NaN,NaN


### Merging port calls data with the covid data for the world

1.	In order to analyze the relation between the <u>world port calls and covid case</u>, we need to merge both data. This involves grouping the the covid data by date to obtain the sum of all the covid cases.
2.	The merging aims to include all the data from both port calls data and covid case hence we can see the overlapping data and missing data point to ease the analysis

In [19]:
df_port_calls_world = df_port_calls[df_port_calls.country == 'World']
df_covid_world = df_covid_new.groupby('date').sum()
df_covid_world = df_covid_world.drop(['2020-01-31','2023-01-31'])

portcalls_covid_world = pd.merge(df_port_calls_world, df_covid_world, on=['date'], how='outer')
portcalls_covid_world

,period,year,country,vessel_type,num_port_calls,date,new_cases,cumulative_cases
0,2018S01,2018,World,All ships,1984908,2018-07-31,NaN,NaN
1,2018S01,2018,World,Passenger ships,1053697,2018-07-31,NaN,NaN
2,2018S01,2018,World,Liquid bulk carriers,245147,2018-07-31,NaN,NaN
3,2018S01,2018,World,Container ships,226063,2018-07-31,NaN,NaN
4,2018S01,2018,World,Dry breakbulk carriers,211031,2018-07-31,NaN,NaN
...,...,...,...,...,...,...,...,...
76,2022S01,2022,World,Dry breakbulk carriers,210158,2022-07-31,198095204.0,9.040880e+10
77,2022S01,2022,World,Dry bulk carriers,138318,2022-07-31,198095204.0,9.040880e+10
78,2022S01,2022,World,Roll-on/ roll-off ships,88572,2022-07-31,198095204.0,9.040880e+10
79,2022S01,2022,World,Liquefied petroleum gas carriers,29257,2022-07-31,198095204.0,9.040880e+10


### Merging port data with the covid data for the world


1.	In order to analyze the relation between <u>median time between port calls and covid case</u>, we need to merge both data. This involves grouping the the covid data by date to obtain the sum of all the covid cases.
2.	The merging aims to include all the data from both port data and covid case hence we can see the overlapping data and missing data point to ease the analysis.

In [20]:
df_ports_world = df_port[df_port.country == 'World']
df_covid_world = df_covid_new.groupby('date').sum()
df_covid_world = df_covid_world.drop(['2020-01-31','2023-01-31'])


port_covid_world = pd.merge(df_ports_world, df_covid_world, on=['date'], how='outer')
port_covid_world

,period,year,country,vessel_type,median_time_in_port,avg_age_of_vessel,avg_size_of_vessel,avg_dwt,date,new_cases,cumulative_cases
0,2018S01,2018,World,All ships,0.9700,18,15222,24074.0,2018-07-31,NaN,NaN
1,2018S01,2018,World,Passenger ships,NaN,21,8978,NaN,2018-07-31,NaN,NaN
2,2018S01,2018,World,Liquid bulk carriers,0.9400,13,15470,26871.0,2018-07-31,NaN,NaN
3,2018S01,2018,World,Container ships,0.6900,13,38405,NaN,2018-07-31,NaN,NaN
4,2018S01,2018,World,Dry breakbulk carriers,1.1200,19,5455,7413.0,2018-07-31,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
76,2022S01,2022,World,Dry breakbulk carriers,1.1938,21,5571,7604.0,2022-07-31,198095204.0,9.040880e+10
77,2022S01,2022,World,Dry bulk carriers,2.2306,14,32735,58640.0,2022-07-31,198095204.0,9.040880e+10
78,2022S01,2022,World,Roll-on/ roll-off ships,NaN,17,25706,10319.0,2022-07-31,198095204.0,9.040880e+10
79,2022S01,2022,World,Liquefied petroleum gas carriers,1.0292,16,10726,11986.0,2022-07-31,198095204.0,9.040880e+10


### Merging the port data and the performance data

To analyze the performance data with the port data, the port calls data frame, port data frame and the performance index data frame are combined together for visualisation purposes.</br> 
In order to attain this, all 3 data frames had to matched with common column properties.</br>
Firstly, we drop all the drop corresponding to to 2018, 2019 and 2022 from the port data and the port calls data to match with the data contained in the performance data. the databases are then merged and the duplicates are dropped to clear out the data.</br>
Further manipulation before megering the port data, port calls data and the performance data.</br>
1. The port data contained 2 values of 'median time in ports' for the 2 semesters in a year. while, in the performance data frame, the data was provided yearly. Therefore, in order to standardize the data sets, the mean of the 'median time in ports' for the year was taken.
2. Similarly, the port calls data contained 2 values of 'port calls' for the 2 semesters in a year, while, in the performance data frame, the data was provided yearly. Therefore, in order to standardize the data sets, the mean of the 'number of port calls' for the year was taken.
3. Lastly, in the performance data frame, we take the mean of all the 'statistic approach rank' and 'index value' for all the ports in a country.</br>

With these modified data sets, we can now analyze the impact of the number of port calls and the median time in ports with the port performance.


In [21]:
# Dropping year 2018, 2019 and 2022 from the duplicate port calls data
df_call = df_port_calls
df_call = df_call.drop(df_call[df_call["year"] == 2018].index)
df_call = df_call.drop(df_call[df_call["year"] == 2019].index)
df_call = df_call.drop(df_call[df_call["year"] == 2022].index)

# Dropping year 2018, 2019 and 2022 from the duplicate port data
df_port_time = df_port
df_port_time = df_port_time.drop(df_port_time[df_port_time["year"] == 2018].index)
df_port_time = df_port_time.drop(df_port_time[df_port_time["year"] == 2019].index)
df_port_time = df_port_time.drop(df_port_time[df_port_time["year"] == 2022].index)

# Grouping the port_calls data and finding the the mean of the 'number of port calls' for each vessel type for each each for a country.
df_call = df_call.groupby(['country', 'vessel_type', 'year']).agg({'num_port_calls': ['mean']}).reset_index()
df_call.columns = ['country', 'vessel_type', 'year', 'num_port_calls']

# Grouping the port data and finding the the mean of the 'median time in ports' for each vessel type for each each for a country.
df_port_time = df_port_time.groupby(['country', 'vessel_type', 'year']).agg({'median_time_in_port': ['mean']}).reset_index()
df_port_time.columns = ['country', 'vessel_type', 'year', 'median_time_in_port']

# Grouping the performance data and finding the the mean of the 'index_value' and 'statistic approach rank' for each vessel type for each each for a country.
df_performane_new = df_performance.groupby(['country', 'year']).agg({'index_value': ['mean'], 'statistic_approach_rank': ['mean']}).reset_index()
df_performane_new.columns = ['country', 'year', 'index_value', 'statistic_approach_rank']

# Merging port data, port calls data and the performance data.
port_time_performance = pd.merge(df_port_time, df_performane_new, on=['country', "year"], how = "inner")
port_performance = pd.merge(df_call, port_time_performance, on=['country', 'year', 'vessel_type'], how = "inner")


port_performance = port_performance.drop(port_performance[port_performance["vessel_type"] != "Container ships"].index)
port_performance = port_performance.drop(port_performance[port_performance["country"] == "World"].index)

# displaying result
port_performance.head()

,country,vessel_type,year,num_port_calls,median_time_in_port,index_value,statistic_approach_rank
2,Australia,Container ships,2020,1829.0,1.43140,-21.000000,243.000000
3,Australia,Container ships,2021,1791.5,1.57775,-12.952000,282.000000
20,Canada,Container ships,2020,1020.5,1.74285,7.400000,145.800000
21,Canada,Container ships,2021,1029.5,1.79550,-50.782200,254.600000
36,China,Container ships,2020,37206.5,0.62150,26.811235,127.823529


## Section 3.3: Filtering merged data for data visualization

In this section, we filter out the necessary data sets created in the previous section to compare and analyze.

### Filtering geo_port data
Here, 'all vessels' are taken for analyzing the 'median time in port' for the world.</br>
Also, the date column had to be modified so that it would be readable by plotly

In [22]:
geo_port_all = geo_port[
    (geo_port.vessel_type == 'All ships')
]

for i in range(len(geo_port_all)):
    k=datetime.strptime(str(geo_port_all.iloc[i,10]), "%Y-%m-%d %H:%M:%S").date()
    geo_port_all.iloc[i,10]=k

C:\Users\john_\AppData\Local\Temp\ipykernel_40356\1666149402.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Peaks and Valleys 
I'm going to find common peaks for three kinds of port data and for new covid cases thru the years. All for the country of the USA.

First, we would use a find-my-peaks algorithm to define how to find the peaks and the valleys.
Then, we would plot graphs of the peaks and valleys.
The peaks_dict is necessary to create a list of the peaks to plot. We will also include the dates_dict containing the dates of the peaks and valleys. We do this because eventually we want to be able to compare dates and not numbers.

Even when we have the plots, we're going to use code to look for common peaks and valleys as well.

A margin is needed to also report peaks that differ by a semester or two, to be able to e.g. see the effect of higher covid cases on number of port calls.
We loop over the peaks of both the activity variables (port data or covid data)
We loop over all the numbers in the margin range (so if months=4, I loop over 1, 2, 3, and 4)
In the loop, it is checked whether the two peaks are on the same date, or if peak A equals peak B if you add or distract the month number
If one of those is the case, peak A is added to one of three lists: precise, before or after. These show whether peak B is either in the same semester, in the margin before or in the margin after that semester
Then we print those dictionary lists in nicely formulated sentences.
Additional information is also given in the comments in the code.

In [23]:
# first, I'm going to define a function to be able to find peaks and valleys in the data sets that we have
def data_highs(data, activity, **kwargs):

    diff_1 = data[activity].diff(periods = -1)
    diff_2 = data[activity].diff(periods = 1)
    
    peaks = []
    for i in range(len(diff_1)):
        if diff_1[i] > 0 and diff_2[i] > 0:
            peaks.append(int(i))          
            
    return peaks

# And do the same for the valleys
def data_lows(data, activity, **kwargs):

    diff_1 = data[activity].diff(periods = -1)
    diff_2 = data[activity].diff(periods = 1)

    valleys = []
    for i in range(len(diff_1)):
        if diff_1[i] < 0 and diff_2[i] < 0:
            valleys.append(int(i))          
            
    return valleys

In [24]:
# I create a dataframe that contains only the data for the USA and all ships and reset the indices for it
geo_port_all_vessels = pd.merge(port_covid, df_port_calls, on=['country','date', 'vessel_type', 'year'], how='outer')
geo_port_all_vessels = geo_port_all_vessels.dropna(subset=['new_cases'])
geo_port_all_vessels = geo_port_all_vessels[(geo_port_all_vessels.country == 'United States of America')]
geo_port_all_vessels = geo_port_all_vessels[(geo_port_all_vessels.vessel_type == 'All ships')]
geo_port_all_vessels.reset_index(inplace=True)

In [25]:
# Variables from COVID data 
activity_1 = 'new_cases'

# Variables from Maritime data 
activity_2 = 'median_time_in_port'
activity_3 = 'num_port_calls'
activity_4 = 'avg_size_of_vessel'
activity_5 = 'avg_age_of_vessel'

country_1 = 'United States of America'

activities_story_1 = [activity_4, activity_3] #this order is set so that I can get the correct peaks/valleys for median time later down 
activities_story_2 = [activity_5, activity_2]
activities_story_3 = [activity_1]
activities_story_4 = [activity_2]

### Spearmans corelation

#### US, Japan and China

First, we select US and Japan as a representation of western and eastern country. We proceed to calculate the spearman correlation coefficient and p-value using the scipcy library.
In order to use this method, all NaN or Nill data must be first filtered out and then the calculation of spearman correlation coefficient can proceed to

To continue with the verification of the hypothesis, we select another eastern country which is China and again calculate the Spearman correlation coefficient using the scipy library.

In [26]:
port_covid_new = port_covid[port_covid.vessel_type == 'All ships']
port_covid_new = port_covid_new.dropna(subset=['new_cases'])
port_covid_new = port_covid_new.dropna(subset=['vessel_type'])
country_list_1 = ['United States of America', "Japan", "China"]
n1 = len(country_list_1)

correlation_c1 = []
p_value_1 = []
for i in range(0, n1):
    correlation_c1.append(scipy.stats.spearmanr(
                        port_covid_new[
                            port_covid_new["country"] == country_list_1[i]]['new_cases'], 
                            port_covid_new[port_covid_new["country"] == country_list_1[i]]['median_time_in_port'
                            ]
                        )[0])
    p_value_1.append(scipy.stats.spearmanr(
                        port_covid_new[
                            port_covid_new["country"] == country_list_1[i]]['new_cases'],
                            port_covid_new[port_covid_new["country"] == country_list_1[i]]['median_time_in_port'
                            ]
                        )[1])




Lastly, to conclude our analysis, we calculate all available country spearman correlation coefficient and put it into a new dataframe. Using this dataframe, we are able to plot all the data in a form of geoplot using plotly express in order to ease presentation of all data.

In [27]:
country_list_2 = port_covid_new.country.unique()
n2 = len(country_list_2)

correlation_c2 = []
p_value_2 =[]
for i in range(0, n2):
    correlation_c2.append(scipy.stats.spearmanr(
                        port_covid_new[
                            port_covid_new["country"] == country_list_2[i]]['new_cases'], 
                            port_covid_new[port_covid_new["country"] == country_list_2[i]]['median_time_in_port'
                            ]
                        )[0])
    p_value_2.append(scipy.stats.spearmanr(
                        port_covid_new[
                            port_covid_new["country"] == country_list_2[i]]['new_cases'],
                            port_covid_new[port_covid_new["country"] == country_list_2[i]]['median_time_in_port'
                            ]
                        )[1])
